In [1]:
import boto3
import json

In [2]:
# get current accout and region
account = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ds_nb_exec_role = f'arn:aws:iam::{account}:role/abc.dev.all.sagemaker.data-scientist-exec.role'
ds_nb_resource = f'arn:aws:sagemaker:{region}:{account}:notebook-instance/*'
ds_policy_name = 'abc.dev.us-west-2.data-scientist.create-nb.policy'

print(f'{ds_nb_exec_role} \n{ds_nb_resource}')

arn:aws:iam::476271697919:role/abc.dev.all.sagemaker.data-scientist-exec.role 
arn:aws:sagemaker:us-west-2:476271697919:notebook-instance/*


In [4]:
# Create a data scientist policy
# String condition operators: 
# https://docs.aws.amazon.com/IAM/latest/UserGuide/reference_policies_elements_condition_operators.html#Conditions_String
# https://docs.aws.amazon.com/IAM/latest/UserGuide/access_iam-tags.html
# https://docs.aws.amazon.com/IAM/latest/UserGuide/list_amazonsagemaker.html

ds_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": ds_nb_exec_role
        },
        {
            "Effect": "Allow",
            "Action": "sagemaker:CreateNotebookInstance",
            "Resource": ds_nb_resource,
            "Condition": {
                "StringEquals": {
                    "aws:ResourceTag/abc.cost-center": "dept1"
                },
                "ForAllValues:StringEquals": {
                    "aws:TagKeys": [
                        "abc.owner-name",
                        "abc.cost-center",
                        "abc.project"
                    ]
                },
                "ForAnyValue:StringEquals": {
                    "sagemaker:InstanceTypes": [
                        "ml.t2.medium",
                        "ml.t3.medium"
                    ]
                }
            }
        }
    ]
}

In [6]:
# Create IAM client
iam = boto3.client('iam')

response = iam.create_policy(
  PolicyName=ds_policy_name,
  PolicyDocument=json.dumps(ds_policy)
)
print(response)
policy_arn = response['Policy']['Arn']
policy_arn

{'Policy': {'PolicyName': 'abc.dev.us-west-2.data-scientist.create-nb.policy', 'PolicyId': 'ANPAW5ZABL77XDDBFN4UP', 'Arn': 'arn:aws:iam::476271697919:policy/abc.dev.us-west-2.data-scientist.create-nb.policy', 'Path': '/', 'DefaultVersionId': 'v1', 'AttachmentCount': 0, 'PermissionsBoundaryUsageCount': 0, 'IsAttachable': True, 'CreateDate': datetime.datetime(2020, 8, 19, 14, 18, 40, tzinfo=tzlocal()), 'UpdateDate': datetime.datetime(2020, 8, 19, 14, 18, 40, tzinfo=tzlocal())}, 'ResponseMetadata': {'RequestId': 'e2d1b259-9d62-4c9b-a45b-5a29f239c3a7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e2d1b259-9d62-4c9b-a45b-5a29f239c3a7', 'content-type': 'text/xml', 'content-length': '833', 'date': 'Wed, 19 Aug 2020 14:18:39 GMT'}, 'RetryAttempts': 0}}


'arn:aws:iam::476271697919:policy/abc.dev.us-west-2.data-scientist.create-nb.policy'

# create data scient execution role

In [7]:
ds_role_name = 'abc.dev.us-west-2.data-scientist.basic-execution.role'

In [8]:
# https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_create_policy-examples.html
ds_assume_role_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"AWS": account},
            "Action": [
                "sts:AssumeRole"
            ]
        }
    ]
}

In [19]:
response = iam.create_role(
    RoleName=ds_role_name,
    AssumeRolePolicyDocument=json.dumps(ds_assume_role_policy)
)

In [22]:
roleArn=response['Role']['Arn']
print(roleArn)

arn:aws:iam::476271697919:role/abc.dev.us-west-2.data-scientist.basic-execution.role


In [23]:
response = iam.attach_role_policy(RoleName=ds_role_name, PolicyArn=policy_arn)
response

{'ResponseMetadata': {'RequestId': '61c2dad2-4f62-4b12-b50d-b936a42946b7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '61c2dad2-4f62-4b12-b50d-b936a42946b7',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Wed, 19 Aug 2020 14:26:15 GMT'},
  'RetryAttempts': 0}}

# Create SageMaker Notebook Execution Role

In [31]:
nb_role_name = 'abc.dev.all.sagemaker.data-scientist-exec.role'
nb_assume_role_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": { "Service": ["sagemaker.amazonaws.com"] },
            "Action": [
                "sts:AssumeRole"
            ]
        }
    ]
}
response = iam.create_role(RoleName=nb_role_name,
    AssumeRolePolicyDocument=json.dumps(nb_assume_role_policy))

# sagemaker_nb_role = f'arn:aws:iam::{account}:role/abc.dev.all.sagemaker.data-scientist-exec.role'
response


{'Role': {'Path': '/',
  'RoleName': 'abc.dev.all.sagemaker.data-scientist-exec.role',
  'RoleId': 'AROAW5ZABL775OPBUC2PY',
  'Arn': 'arn:aws:iam::476271697919:role/abc.dev.all.sagemaker.data-scientist-exec.role',
  'CreateDate': datetime.datetime(2020, 8, 19, 14, 45, 11, tzinfo=tzlocal()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Effect': 'Allow',
     'Principal': {'Service': ['sagemaker.amazonaws.com']},
     'Action': ['sts:AssumeRole']}]}},
 'ResponseMetadata': {'RequestId': '4925f1e8-d013-40f1-91c5-fd1d4892cf1a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4925f1e8-d013-40f1-91c5-fd1d4892cf1a',
   'content-type': 'text/xml',
   'content-length': '861',
   'date': 'Wed, 19 Aug 2020 14:45:11 GMT'},
  'RetryAttempts': 0}}

In [32]:
sagemaker_nb_role = response['Role']['Arn']
sagemaker_nb_role

'arn:aws:iam::476271697919:role/abc.dev.all.sagemaker.data-scientist-exec.role'

## Verify the role

In [33]:
sts = boto3.client('sts')
roleSSName = 'ds-session'
response = sts.assume_role(RoleArn=roleArn, RoleSessionName=roleSSName)
ds_credentials = response['Credentials']

# use data-science credentials to create sagemaker
ds_session = boto3.Session(aws_access_key_id=ds_credentials['AccessKeyId'],
    aws_secret_access_key=ds_credentials['SecretAccessKey'],
    aws_session_token=ds_credentials['SessionToken'],
    region_name=region)

In [34]:
ds_sm = ds_session.client(service_name='sagemaker')

In [35]:
from datetime import datetime
now = datetime.now().strftime("-%Y-%m-%d-%H-%M-%S") # current date and time

nb_name = 'demo' + now

In [36]:
response = ds_sm.create_notebook_instance(NotebookInstanceName=nb_name,
                                          InstanceType='ml.t3.medium',
                                          RoleArn=sagemaker_nb_role,
                                          Tags=[{'Key':'abc.owner-name', 'Value':'beyoung'},
                                                {'Key':'abc.cost-center', 'Value': 'dept1'},
                                                {'Key':'abc.project', 'Value': 'forecast'},                                                
                                          ])
response

ClientError: An error occurred (AccessDeniedException) when calling the CreateNotebookInstance operation: User: arn:aws:sts::476271697919:assumed-role/abc.dev.us-west-2.data-scientist.basic-execution.role/ds-session is not authorized to perform: sagemaker:CreateNotebookInstance on resource: arn:aws:sagemaker:us-west-2:476271697919:notebook-instance/demo-2020-08-19-14-46-10

# Delete Role

In [18]:
iam.detach_role_policy(RoleName=ds_role_name, PolicyArn=policy_arn)
response = iam.delete_role(RoleName=ds_role_name)
response

{'ResponseMetadata': {'RequestId': '75d7005a-cd8c-4b20-bad8-7961afb86bbf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '75d7005a-cd8c-4b20-bad8-7961afb86bbf',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Wed, 19 Aug 2020 14:25:16 GMT'},
  'RetryAttempts': 0}}